# Write discriminator and generator for model

Start by loading data

In [1]:
import pandas as pd
import tables

In [2]:
df = pd.read_hdf('/storage/er_predict.h5', 'table')

In [3]:
df.shape[0]

173447

In [4]:
import tensorflow as tf
#import tensorflow.keras as keras

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
!pip install keras

    100% |################################| 307kB 3.2MB/s ta 0:00:01
    100% |################################| 276kB 3.6MB/s ta 0:00:01
    100% |################################| 51kB 10.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for pyyaml ... done
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
Successfully built pyyaml
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation

Using TensorFlow backend.


In [9]:
import numpy as np

In [10]:
df2 = df.iloc[0:10000,].drop(["regdato"], axis=1)

In [11]:
df2.dtypes

ansatte_antall        float64
forradrkommnr_0101      uint8
forradrkommnr_0104      uint8
forradrkommnr_0105      uint8
forradrkommnr_0106      uint8
forradrkommnr_0111      uint8
forradrkommnr_0118      uint8
forradrkommnr_0119      uint8
forradrkommnr_0121      uint8
forradrkommnr_0122      uint8
forradrkommnr_0123      uint8
forradrkommnr_0124      uint8
forradrkommnr_0125      uint8
forradrkommnr_0127      uint8
forradrkommnr_0128      uint8
forradrkommnr_0135      uint8
forradrkommnr_0136      uint8
forradrkommnr_0137      uint8
forradrkommnr_0138      uint8
forradrkommnr_0211      uint8
forradrkommnr_0213      uint8
forradrkommnr_0214      uint8
forradrkommnr_0215      uint8
forradrkommnr_0216      uint8
forradrkommnr_0217      uint8
forradrkommnr_0219      uint8
forradrkommnr_0220      uint8
forradrkommnr_0221      uint8
forradrkommnr_0226      uint8
forradrkommnr_0227      uint8
                       ...   
nkode1_96.020           uint8
nkode1_96.030           uint8
nkode1_96.

In [12]:
def traningdata(df, n, rep):
    X = np.empty(shape=[ rep, n, df.shape[1] ] )
    y = np.empty( shape= [ rep ] )
    for i in range(0, rep):
        repeat = np.random.binomial(1, 0.5, 1)[0]
        X[i, :, :] = df.sample(n=n, replace=repeat).values
        y[i] = repeat
        
    return (X, y)

In [13]:
X, y = traningdata(df2, 5000, 50)

In [14]:
X.shape

(50, 5000, 1230)

In [15]:
print(y.shape)
print(y[0:10])

(50,)
[1. 1. 0. 0. 1. 1. 1. 1. 1. 0.]


In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [18]:
X_train.shape[2]

1230

In [28]:
m = Sequential()
m.add(Conv2D(256, (5, 5), padding='valid', input_shape=(X_train.shape[1], X_train.shape[2], 1), activation='relu'))
#m.add(Conv2D(128, 5, strides=(2, 2), padding='valid', activation='relu'))
m.add(MaxPool2D((2,2)))
m.add(Conv2D(128, 5, strides=(2, 2), padding='valid', activation='relu'))
m.add(MaxPool2D((2,2)))
m.add(Flatten())
m.add(Dense(64, activation='relu'))
m.add(Dense(1))
m.add(Activation('sigmoid'))

m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 4996, 1226, 256)   6656      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 2498, 613, 256)    0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 1247, 305, 128)    819328    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 623, 152, 128)     0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 12121088)          0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                775749696 
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
__________

In [29]:
m.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [21]:
X_train.shape

(40, 5000, 1230)

In [23]:
X_train2 = X_train.reshape((40, 5000, 1230, 1))
X_test2 = X_test.reshape((10, 5000, 1230, 1))
X_train2.shape

(40, 5000, 1230, 1)

In [24]:
X_test2.shape

(10, 5000, 1230, 1)

In [30]:
history = m.fit(X_train2, y_train,
                    batch_size=10,
                    epochs=5,
                    verbose=1)

Epoch 1/5


ResourceExhaustedError: OOM when allocating tensor with shape[12121088,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: dense_3/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=5287414, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_3/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense_3/random_uniform/RandomUniform', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_in

In [27]:
del m